<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-06-06 13:44:25
-------------------
qualified stocks: 93
with latest results: 31
still star stocks: 17
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  1.64 L
Current:  1.33 C
-------------------
Today PnL: 66.05 K (0.5%)
Current PnL: -15.64 L (-10.9%)
CY Booked + Current PnL: -10.65 L (-7.42%)
-------------------
Total profit:  4.53 L
Total loss:  -20.17 L
-------------------
Total Booked + Current PnL: 16.33 L (14.01%)
Total Booked PnL: 31.97 L (27.43%)
Curr Year Booked PnL: 4.99 L (3.75%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.12 C
Est FTT PnL: 78.53 L (59.06%)
Deployed:  1.17 C
Current:  1.33 C
CAGR/XIRR %: 9.38%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,DLF,714.31,885.0,-5.31,H-LC,92.47,288814.0,52377.0,4130.0,5.67,22.15,1.43,23.90,30.0,12.68,2.20,43.50,AR,NTT,REALTY
50,LICI,785.05,983.0,-3.65,H-LC,90.32,236989.0,42297.0,6802.0,-0.10,21.72,2.87,25.21,31.0,6.22,1.80,31.05,XY25,NTT,INSURANCE
4,ADANIPORTS,1103.69,1583.0,4.33,M-LC,63.44,104597.0,26235.0,7792.0,1.13,33.48,7.45,43.43,70.0,3.37,0.80,39.90,XY24,NTT,MISC
62,RELIANCE,1291.83,1526.0,5.74,X-LC,24.73,157309.0,16500.0,9030.0,0.06,11.72,5.74,18.13,19.0,1.83,1.20,24.21,XY25,NTT,REFINERIES
85,UNIONBANK,123.87,163.0,-0.31,M-LC,93.55,174143.0,33303.0,11180.0,0.40,23.65,6.42,31.59,79.0,2.98,1.33,51.27,XY24,NTT,BANKS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,UJJIVANSFB,52.77,60.00,48.19,M-SC,81.72,128709.0,-13770.0,33297.0,7.12,-9.66,25.87,13.70,221.0,-0.41,0.98,51.91,X40N,NTT,BANKS
26,DLF,714.31,885.00,-5.31,H-LC,92.47,288814.0,52377.0,4130.0,5.67,22.15,1.43,23.90,30.0,12.68,2.20,43.50,AR,NTT,REALTY
64,ROUTE,1558.40,2249.26,-38.28,M-SC,40.86,66614.0,-36240.0,81835.0,4.72,-35.23,122.85,44.33,172.0,-0.44,0.51,10.62,SR,ATH,IT
63,REPCOHOME,515.07,880.00,-55.29,H-SC,57.53,165727.0,-39271.0,184520.0,4.44,-19.16,111.34,70.85,121.0,-0.21,1.26,33.78,XY24,NTT,FINANCE
46,JPPOWER,18.73,26.20,-38.83,L-SC,88.17,124576.0,-17266.0,73836.0,4.44,-12.17,59.27,39.88,234.0,-0.23,0.95,26.25,XY24,NTT,POWER


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,VALIANTORG,512.64,838.0,-362.63,H-SC,100.00,111657.0,-21629.0,106219.0,-3.56,-16.23,95.13,63.47,130.0,-0.20,0.85,87.16,XR,NTT,CHEMICALS
60,RBLBANK,230.61,388.0,35.28,M-SC,52.69,210200.0,-24100.0,184009.0,-2.63,-10.29,87.54,68.25,222.0,-0.13,1.60,38.42,XY24,NTT,BANKS
73,SONACOMS,578.05,1006.0,-30.24,M-MC,31.18,90169.0,-10990.0,85886.0,-2.43,-10.86,95.25,74.03,155.0,-0.13,0.69,26.72,AR,BTT,AUTO
72,SIS,477.00,528.0,2223.89,M-SC,73.12,62917.0,-18173.0,26840.0,-1.71,-22.41,42.66,10.69,201.0,-0.68,0.48,27.51,X40N,NTT,MISC
56,PFIZER,4245.50,6318.3,13.55,H-SC,97.85,131112.0,33466.0,14213.0,-1.66,34.27,10.84,48.82,134.0,2.35,1.00,45.65,X40,ATH,PHARMA


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,INFY,1461.46,2275.00,-13.14,X-LC,36.56,278470.0,16869.0,128765.0,0.09,6.45,46.24,55.67,1.0,0.13,2.12,12.95,X40,BTT,IT
80,TCS,3794.03,4998.00,-19.05,X-LC,12.90,263328.0,-32606.0,126529.0,0.15,-11.02,48.05,31.73,2.0,-0.26,2.00,5.40,X40,BTT,IT
51,LTIM,5564.16,7197.33,-2.50,X-LC,79.57,187560.0,-12750.0,71535.0,0.55,-6.37,38.14,29.35,3.0,-0.18,1.43,29.58,X200,ATH,IT
7,ASIANPAINT,2961.56,4250.00,-23.26,X-LC,4.30,152708.0,-48678.0,136292.0,0.10,-24.17,89.25,43.51,5.0,-0.36,1.16,5.33,X40,BTT,PAINTS
35,HINDUNILVR,2413.81,2723.00,-15.17,X-LC,18.28,266448.0,-3899.0,38528.0,0.11,-1.44,14.46,12.81,6.0,-0.10,2.03,10.27,XY25,NTT,FMCG


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,PFIZER,4245.50,6318.30,13.55,H-SC,97.85,131112.0,33466.0,14213.0,-1.66,34.27,10.84,48.82,134.0,2.35,1.00,45.65,X40,ATH,PHARMA
57,POLYCAB,5005.95,7541.35,8.17,H-MC,29.03,102697.0,17596.0,25510.0,1.01,20.68,24.84,50.65,90.0,0.69,0.78,29.09,X40N,ATH,CABLES
85,UNIONBANK,123.87,163.00,-0.31,M-LC,93.55,174143.0,33303.0,11180.0,0.40,23.65,6.42,31.59,79.0,2.98,1.33,51.27,XY24,NTT,BANKS
4,ADANIPORTS,1103.69,1583.00,4.33,M-LC,63.44,104597.0,26235.0,7792.0,1.13,33.48,7.45,43.43,70.0,3.37,0.80,39.90,XY24,NTT,MISC
50,LICI,785.05,983.00,-3.65,H-LC,90.32,236989.0,42297.0,6802.0,-0.10,21.72,2.87,25.21,31.0,6.22,1.80,31.05,XY25,NTT,INSURANCE


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SAIL,130.64,228.00,104.28,M-MC,87.10,231953.0,6991.0,160674.0,1.19,3.11,69.27,74.53,170.0,0.04,1.77,34.67,XY24,BTT,STEEL
52,LUXIND,1448.19,2442.00,38.95,M-SC,70.97,143934.0,6356.0,88059.0,-0.79,4.62,61.18,68.62,183.0,0.07,1.10,24.78,X40N,NTT,TEXTILES
56,PFIZER,4245.50,6318.30,13.55,H-SC,97.85,131112.0,33466.0,14213.0,-1.66,34.27,10.84,48.82,134.0,2.35,1.00,45.65,X40,ATH,PHARMA
57,POLYCAB,5005.95,7541.35,8.17,H-MC,29.03,102697.0,17596.0,25510.0,1.01,20.68,24.84,50.65,90.0,0.69,0.78,29.09,X40N,ATH,CABLES
18,CAMS,3643.00,5250.99,6.22,H-SC,84.95,119524.0,17520.0,27502.0,0.48,17.18,23.01,44.14,112.0,0.64,0.91,36.21,XR,ATH,MISC


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CAMS,3643.00,5250.99,6.22,H-SC,84.95,119524.0,17520.0,27502.0,0.48,17.18,23.01,44.14,112.0,0.64,0.91,36.21,XR,ATH,MISC
12,BANKINDIA,113.49,190.00,-28.22,M-MC,72.04,141590.0,12438.0,74632.0,0.14,9.63,52.71,67.42,156.0,0.17,1.08,37.71,XR,NTT,BANKS
39,INDIAMART,2327.09,4871.06,-51.09,H-SC,54.84,127613.0,4277.0,130548.0,-0.41,3.47,102.30,109.32,108.0,0.03,0.97,26.07,AR,ATH,MISC
92,WIPRO,243.46,420.00,-10.16,M-LC,32.26,153878.0,2933.0,106530.0,0.10,1.94,69.23,72.51,68.0,0.03,1.17,9.81,XR,NTT,IT
1,AARTIIND,487.04,919.00,38.87,M-SC,51.61,79759.0,-1577.0,73713.0,-0.84,-1.94,92.42,88.69,205.0,-0.02,0.61,34.50,XR,NTT,CHEMICALS


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MASFIN,326.60,399.5,-22.31,H-SC,66.67,88530.0,-9450.0,31322.0,3.65,-9.64,35.38,22.32,127.0,-0.30,0.67,28.40,XR,ATH,FINANCE
4,ADANIPORTS,1103.69,1583.0,4.33,M-LC,63.44,104597.0,26235.0,7792.0,1.13,33.48,7.45,43.43,70.0,3.37,0.80,39.90,XY24,NTT,MISC
87,VALIANTORG,512.64,838.0,-362.63,H-SC,100.00,111657.0,-21629.0,106219.0,-3.56,-16.23,95.13,63.47,130.0,-0.20,0.85,87.16,XR,NTT,CHEMICALS
84,UJJIVANSFB,52.77,60.0,48.19,M-SC,81.72,128709.0,-13770.0,33297.0,7.12,-9.66,25.87,13.70,221.0,-0.41,0.98,51.91,X40N,NTT,BANKS
56,PFIZER,4245.50,6318.3,13.55,H-SC,97.85,131112.0,33466.0,14213.0,-1.66,34.27,10.84,48.82,134.0,2.35,1.00,45.65,X40,ATH,PHARMA


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,VALIANTORG,512.64,838.0,-362.63,H-SC,100.00,111657.0,-21629.0,106219.0,-3.56,-16.23,95.13,63.47,130.0,-0.20,0.85,87.16,XR,NTT,CHEMICALS
8,ASIANTILES,75.41,137.0,6583.33,M-SC,98.92,73359.0,-20451.0,97069.0,2.77,-21.80,132.32,81.67,214.0,-0.21,0.56,42.06,XR,NTT,CERAMICS
56,PFIZER,4245.50,6318.3,13.55,H-SC,97.85,131112.0,33466.0,14213.0,-1.66,34.27,10.84,48.82,134.0,2.35,1.00,45.65,X40,ATH,PHARMA
81,TITAGARH,1097.23,1548.0,-6.68,H-SC,96.77,164124.0,-27891.0,106779.0,0.42,-14.53,65.06,41.08,128.0,-0.26,1.25,38.91,XY24,NTT,ENGINEERING
74,STARHEALTH,551.56,761.0,3.93,M-SC,95.70,169957.0,-30259.0,106291.0,-0.49,-15.11,62.54,37.97,195.0,-0.28,1.29,36.46,XY24,NTT,INSURANCE


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.76
1,25,42.17
2,50,70.53


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    31.68
AR      15.16
XY25    14.60
X40N    13.27
X40     12.40
XR      10.33
X200     1.43
SR       1.15
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.02
X-LC    17.99
M-SC    17.45
H-LC     9.30
H-MC     8.50
M-LC     7.72
M-MC     5.53
X-MC     5.50
X-SC     2.75
L-SC     2.32
L-LC     1.12
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.41,-1.69,33.54
IT,10.18,-16.63,79.98
BANKS,9.75,-9.52,59.87
MISC,6.48,-27.17,83.17
FINANCE,5.89,-23.46,82.08
PAINTS,5.28,-18.71,53.84
INSURANCE,5.23,3.22,25.69
ELECTRICAL,5.09,-5.10,50.69
HEALTHCARE,4.36,-3.94,34.95


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2815868.0,26
AR,1232050.0,14
X40N,1204850.0,17
XR,1095642.0,14
X40,778441.0,9
XY25,460981.0,10
SR,194174.0,2
X200,71535.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2209832.0,20
M-SC,2024135.0,21
X-LC,848460.0,11
M-MC,542970.0,5
H-MC,508227.0,8
X-SC,347044.0,4
X-MC,310150.0,5
L-SC,294569.0,4
M-LC,293653.0,6


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      913706.0      6
M-SC       XY24      757870.0      6
H-SC       AR        539130.0      3
           X40N      502466.0      6
X-LC       X40       482255.0      4
M-SC       X40N      464241.0      6
M-MC       XY24      382452.0      3
M-SC       XR        288316.0      4
H-SC       XR        240317.0      4
H-LC       AR        224067.0      5
M-SC       X40       218026.0      2
M-LC       XY24      173610.0      4
H-MC       XY24      173540.0      2
X-LC       AR        165736.0      2
L-SC       XR        159212.0      2
X-MC       XY24      139249.0      2
X-LC       XY25      128934.0      4
M-SC       XY25      119522.0      1
X-SC       X40N      118932.0      2
           XR        115773.0      1
           SR        112339.0      1
H-MC       XR        110862.0      1
M-LC       XR        106530.0      1
M-SC       AR         94325.0      1
X-MC       X40N       93701.0      2
M-MC       AR         85886.0      1
M-SC       SR         81835.0      1
X-MC       XY25       77200.0      1
M-MC       XR         74632.0      1
L-SC       XY24       73836.0      1
H-MC       XY25       72983.0      1
X-LC       X200       71535.0      1
H-MC       X40        63947.0      2
L-SC       AR         61521.0      1
H-MC       AR         61385.0      1
L-LC       XY25       42027.0      1
H-MC       X40N       25510.0      1
H-SC       X40        14213.0      1
M-LC       XY25       13513.0      1
H-LC       XY25        6802.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
